In [2]:
import numpy as np
import pandas as pd

# dataset coronavirus WHO
pd.set_option('max_colwidth', 100)  # Increase column width
data1 = pd.read_excel("/content/WHO_FAQ.xlsx", encoding='utf8')
data1.head()

,Context,Answer
0,What is a coronavirus?,Coronaviruses are a large family of viruses which may cause illness in animals or humans.
1,What is a coronavirus?,"In humans, several coronaviruses are known to cause respiratory infections ranging from the comm..."
2,What is COVID-19?,COVID-19 is the infectious disease caused by the most recently discovered coronavirus. This new ...
3,What are the symptoms of COVID-19?,"The most common symptoms of COVID-19 are fever, tiredness, and dry cough. Some patients may have..."
4,What are the symptoms of COVID-19?,Some people become infected but don’t develop any symptoms and don't feel unwell. Most people (a...


In [3]:
data2 = pd.read_csv("/content/goi.csv", encoding='utf8')
data2.head()

,Question,answer
0,what is corona virus ?,corona viruses are a large family of viruses which may cause illness in animals or humans. in hu...
1,what is covid-19 ?,covid-19 is the infectious disease caused by the most recently discovered corona virus. this new...
2,what are the symptoms of covid-19 ?,"the most common symptoms of covid-19 are fever, tiredness, and dry cough. some patients may have..."
3,how does covid-19 spread,people can catch covid-19 from others who have the virus. the disease can spread from person to ...
4,can the virus that causes covid-19 be transmitted through the air?,studies to date suggest that the virus that causes covid-19 is mainly transmitted through contac...


In [4]:
data2.rename(columns = {'Question':'Context', 'answer':'Answer'} ,
                              inplace = True) 
data2.head()

,Context,Answer
0,what is corona virus ?,corona viruses are a large family of viruses which may cause illness in animals or humans. in hu...
1,what is covid-19 ?,covid-19 is the infectious disease caused by the most recently discovered corona virus. this new...
2,what are the symptoms of covid-19 ?,"the most common symptoms of covid-19 are fever, tiredness, and dry cough. some patients may have..."
3,how does covid-19 spread,people can catch covid-19 from others who have the virus. the disease can spread from person to ...
4,can the virus that causes covid-19 be transmitted through the air?,studies to date suggest that the virus that causes covid-19 is mainly transmitted through contac...


In [7]:
data=pd.concat([data1,data2])

In [18]:
data.shape

(108, 2)

In [8]:
data.tail()

,Context,Answer
18,can humans become infected with the covid-19 from an animal source?,"coronaviruses\tare\ta\tlarge\tfamily\tof\tviruses\tthat\tare\tcommon\tinanimals. occasionally, p..."
19,can i catch covid-19 from my pet?,"while there has been one instance of a dog being infected in hong kong, to date, there is no evi..."
20,how long does the virus survive on surfaces?,"it is not certain how long the virus that causes covid-19 survives on surfaces, but it seems to ..."
21,is it safe to receive a package from any area where covid-19 has been reported?,yes. the likelihood of an infected person contaminating commercialgoods is low and the risk of c...
22,is there anything i should not do?,the following measures are not effective against covid-2019 and can be harmful:smokingwearing mu...


In [9]:
!pip install tensorflow_text

     |████████████████████████████████| 3.0MB 3.4MB/s 


In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import re

def preprocess_sentences(input_sentences):
    return [re.sub(r'(covid-19|covid|corona)', 'coronavirus', input_sentence, flags=re.I) 
            for input_sentence in input_sentences]
        
# Load module containing USE
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

# Create response embeddings
response_encodings = module.signatures['response_encoder'](
        input=tf.constant(preprocess_sentences(data.Answer)),
        context=tf.constant(preprocess_sentences(data.Context)))['outputs']

In [17]:
test_questions = [
    "What to do if I am not feeling good?"
   "Will I be sick if my neighbour is sick?",
   "Will my pet affect covid-19?"
]

# Create encodings for test questions
question_encodings = module.signatures['question_encoder'](
    tf.constant(preprocess_sentences(test_questions))
)['outputs']

# Get the responses
test_responses = data.Answer[np.argmax(np.inner(question_encodings, response_encodings), axis=1)]

test_responses
#pd.DataFrame({'Test Questions': test_questions, 'Test Responses': test_responses})

17    Stay home if you feel unwell. If you have a fever, cough and difficulty breathing, seek medical ...
17    the “incubation period” means the time between catching the virus and beginning to have symptoms...
47                                     Possible animal sources of COVID-19 have not yet been confirmed.  
Name: Answer, dtype: object